In [1]:
import pickle
import os
import tensorflow as tf
import numpy as np
from utils import preprocess_raw_data, get_prediction, create_submission

2025-01-12 17:45:43.335722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-12 17:45:43.346962: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-12 17:45:43.350208: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 17:45:43.359143: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-12 17:45:43.956896: W tensorflow/compiler/tf2

In [2]:
# Preprocess raw data and save it as numpy files
data_path = '../data/'
preprocess_raw_data(data_path)


 Files in the sample subfolder: ['seismicCubes_RFC_fullstack_2023.76113489.npy']

 Files in the sample subfolder: ['seismicCubes_RFC_fullstack_2023.76167908.npy']


In [3]:
# load volumes ids for the submission
with open(os.path.join(data_path, 'file_ids.pkl'), 'rb') as f:
    files_ids = pickle.load(f)

# load numpy files processed
seismic = np.load(os.path.join(data_path, 'seismic_processed.npy'))

In [4]:
# load the pretrained model
path_model = '../model/final_model.keras'
model_inference = tf.keras.models.load_model(path_model, compile=False)

I0000 00:00:1736700348.574259  642624 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1736700348.604806  642624 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1736700348.605004  642624 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1736700348.606871  642624 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [5]:
# Create submission file
for ind, sample_id in enumerate(files_ids):
    sample_id = sample_id.split('/')[-1]
    print('sample_id', sample_id)
    seismic_test = seismic[ind]
    seismic_test = np.expand_dims(seismic_test, axis=0)
    
    # get the prediction for this volume
    prediction = get_prediction(model_inference, seismic_test, model_size_input=128, axis_order=[0, 1, 2], overlap=32)
    
    if np.sum(prediction) < 20_000:
        prediction = prediction*0
    if prediction.shape[0] == 1:
        prediction = prediction[0]

    submission_path = os.path.join(data_path, 'submission.npz')
    # this line creates the submission .npz file and populates it with sample IDs and their fault coordinates
    create_submission(sample_id, prediction, submission_path, append=True)

sample_id 2023-10-05_eff8c5d7


I0000 00:00:1736700352.028819  642884 service.cc:146] XLA service 0x71f84406ff80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1736700352.028835  642884 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2025-01-12 17:45:52.074896: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-01-12 17:45:52.477972: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2025-01-12 17:45:55.154272: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.41GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
I0000 00:00:1736700360.039077  642884 device_compiler.h:188] Compiled cluster using XLA

File not found, new submission will be created.
sample_id 2023-10-05_ef826453
